<a href="https://colab.research.google.com/github/Dhanush-adk/machine_learning/blob/main/assignment_6/assignment_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

time: 12.3 s (started: 2023-12-02 01:00:56 +00:00)


In [3]:
torch.manual_seed(123)

time: 5.73 ms (started: 2023-12-02 01:01:08 +00:00)


In [4]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

100%|██████████| 170498071/170498071 [00:02<00:00, 75240061.14it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
time: 18.5 s (started: 2023-12-02 01:01:08 +00:00)


In [5]:
mean

tensor([0.4914, 0.4822, 0.4465])

time: 31 ms (started: 2023-12-02 01:01:26 +00:00)


In [6]:
std

tensor([0.2470, 0.2435, 0.2616])

time: 4.08 ms (started: 2023-12-02 01:01:26 +00:00)


In [7]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time: 60.8 ms (started: 2023-12-02 01:01:26 +00:00)


In [8]:
device

device(type='cuda')

time: 2.06 ms (started: 2023-12-02 01:01:26 +00:00)


In [9]:
# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
])

time: 570 µs (started: 2023-12-02 01:01:26 +00:00)


In [10]:
cifar10 = datasets.CIFAR10(
    './data', train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


time: 492 ms (started: 2023-12-02 01:01:26 +00:00)


In [11]:
cifar10_val = datasets.CIFAR10(
     './data', train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

time: 352 ms (started: 2023-12-02 01:01:27 +00:00)


In [12]:
first_image, label = cifar10[0]
print(first_image.shape)

torch.Size([3, 32, 32])
time: 11.4 ms (started: 2023-12-02 01:01:27 +00:00)


In [13]:
train_loader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)

time: 544 µs (started: 2023-12-02 01:01:27 +00:00)


In [14]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 10)
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

time: 5.25 s (started: 2023-12-01 22:06:23 +00:00)


In [14]:
def train_model(model, train_loader, test_loader, num_epochs=300, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Testing the model
        model.eval()
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = correct / total

        if epoch % 1 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Test Accuracy: {accuracy * 100:.2f}%')

    # Classification Report
    report = classification_report(all_labels, all_predicted)
    print("Classification Report:\n", report)


time: 1.6 ms (started: 2023-12-02 01:01:27 +00:00)


In [ ]:
train_model(model, train_loader, test_loader, num_epochs=300, lr=0.01)

Epoch 1/300, Loss: 1.7398397800484606, Test Accuracy: 41.89%
Epoch 2/300, Loss: 1.634406721675846, Test Accuracy: 43.21%
Epoch 3/300, Loss: 1.5660125428258953, Test Accuracy: 45.81%
Epoch 4/300, Loss: 1.5045548411645115, Test Accuracy: 45.98%
Epoch 5/300, Loss: 1.4462770357089247, Test Accuracy: 46.90%
Epoch 6/300, Loss: 1.393377955266473, Test Accuracy: 47.19%
Epoch 7/300, Loss: 1.33971855736511, Test Accuracy: 46.97%
Epoch 8/300, Loss: 1.286995179517408, Test Accuracy: 47.02%
Epoch 9/300, Loss: 1.2365475362184637, Test Accuracy: 47.75%
Epoch 10/300, Loss: 1.1836568696027525, Test Accuracy: 48.50%
Epoch 11/300, Loss: 1.1335558970616708, Test Accuracy: 48.34%
Epoch 12/300, Loss: 1.0809114643265938, Test Accuracy: 48.87%
Epoch 13/300, Loss: 1.029821600467062, Test Accuracy: 47.75%
Epoch 14/300, Loss: 0.9790819698774273, Test Accuracy: 48.79%
Epoch 15/300, Loss: 0.9285842062224964, Test Accuracy: 47.34%
Epoch 16/300, Loss: 0.8829675394986886, Test Accuracy: 46.96%
Epoch 17/300, Loss: 0.8

In [16]:
model2 = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 256),
    nn.Tanh(),
    nn.Linear(256, 128),
    nn.Tanh(),
    nn.Linear(128, 10)
).to(device)

time: 5.55 s (started: 2023-12-02 01:01:32 +00:00)


In [17]:
train_model(model2, train_loader, test_loader, num_epochs=200, lr=0.01)

Epoch 1/200, Loss: 1.8592360516961233, Test Accuracy: 39.53%
Epoch 2/200, Loss: 1.689475319664675, Test Accuracy: 42.41%
Epoch 3/200, Loss: 1.6198919219842571, Test Accuracy: 43.72%
Epoch 4/200, Loss: 1.5600317946932518, Test Accuracy: 45.21%
Epoch 5/200, Loss: 1.5012287019310437, Test Accuracy: 46.19%
Epoch 6/200, Loss: 1.444315607434881, Test Accuracy: 46.07%
Epoch 7/200, Loss: 1.3895388086179243, Test Accuracy: 47.66%
Epoch 8/200, Loss: 1.3318402658878652, Test Accuracy: 47.47%
Epoch 9/200, Loss: 1.2729721801142164, Test Accuracy: 48.02%
Epoch 10/200, Loss: 1.2156133362290498, Test Accuracy: 46.54%
Epoch 11/200, Loss: 1.1530542257727527, Test Accuracy: 47.28%
Epoch 12/200, Loss: 1.0960718936746272, Test Accuracy: 48.45%
Epoch 13/200, Loss: 1.0335739719814319, Test Accuracy: 48.03%
Epoch 14/200, Loss: 0.9749154275377363, Test Accuracy: 47.35%
Epoch 15/200, Loss: 0.9124068804109089, Test Accuracy: 44.84%
Epoch 16/200, Loss: 0.854338770292542, Test Accuracy: 48.47%
Epoch 17/200, Loss: 

In [18]:
total_params = sum(p.numel() for p in model2.parameters() if p.requires_grad)
print("Total number of trainable parameters ", total_params,  '\n')

Total number of trainable parameters  1738890 

time: 1.24 ms (started: 2023-12-02 02:09:44 +00:00)
